# Article Spinning

In [20]:
import nltk
import random
import numpy as np
from bs4 import BeautifulSoup
from future.utils import iteritems

In [2]:
positive_reviews = BeautifulSoup(open('/Users/praga/Downloads/electronics/positive.review').read())
positive_reviews = positive_reviews.findAll('review_text')


C:\Users\praga\Anaconda2\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 174 of the file C:\Users\praga\Anaconda2\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


In [3]:
positive_reviews[0]

<review_text>\nI purchased this unit due to frequent blackouts in my area and 2 power supplies going bad.  It will run my cable modem, router, PC, and LCD monitor for 5 minutes.  This is more than enough time to save work and shut down.   Equally important, I know that my electronics are receiving clean power.\n\nI feel that this investment is minor compared to the loss of valuable data or the failure of equipment due to a power spike or an irregular power supply.\n\nAs always, Amazon had it to me in &lt;2 business days\n</review_text>

## Trigrams

In [15]:
# extract trigrams and insert into dictionary
# (w1, w3) is the key, [ w2 ] are the values
trigrams = {}
for review in positive_reviews:
    s = review.text.lower()
    tokens = nltk.tokenize.word_tokenize(s)
    
    for i in xrange(len(tokens) - 2):
        k = (tokens[i], tokens[i+2]) # the tuple is the key.
        if k not in trigrams:
            trigrams[k] = []
        trigrams[k].append(tokens[i+1])

In [32]:
trigrams

{(u'and', u'drums'): [u'the'],
 (u'pennies', u'what'): [u'.'],
 (u'worth', u'hassle'): [u'the'],
 (u'tends', u'pull'): [u'to'],
 (u'small', u'you'): [u'issue'],
 (u'fine', u"'ve"): [u'i'],
 (u'but', u're-set'): [u'the'],
 (u'image', u'voice'): [u'and'],
 (u'i', u'running'): [u"'m", u'am', u"'m", u'was', u"'m"],
 (u'by', u'machine'): [u'the'],
 (u'$', u'than'): [u'more'],
 (u'screen', u'not'): [u'is'],
 (u'have', u'made'): [u'been', u'also'],
 (u'camcorder', u'.'): [u'buff'],
 (u'new', u'to'): [u'addiction'],
 (u'use', u'battery'): [u'the', u'the'],
 (u'by', u'including'): [u'.'],
 (u'want', u'you'): [u'everytime', u'optical'],
 (u'think', u'are'): [u'there', u'these'],
 (u'sink', u'right'): [u'up'],
 (u'picture', u'around'): [u'all'],
 (u'connectors', u'for'): [u'.'],
 (u'things', u'also'): [u'.'],
 (u'was', u'sale'): [u'on'],
 (u',', u'connecting'): [u'but'],
 (u'has', u'uneven'): [u'an'],
 (u'.', u'prefer'): [u'i', u'i'],
 (u'display', u'to'): [u'fonts'],
 (u'at', u'times'): [u'all',

In [21]:
# turn each array of middle-words into a probability vector
trigram_probabilities = {}
for k, words in iteritems(trigrams):
    # create a dictionary of word -> count
    if len(set(words)) > 1:
        # only when there are different possibilities for a middle word
        d = {}
        n = 0
        for w in words:
            if w not in d:
                d[w] = 0
            d[w] += 1
            n += 1
        for w, c in iteritems(d):
            d[w] = float(c) / n
        trigram_probabilities[k] = d

In [27]:
len(trigram_probabilities)

10079

In [28]:
len(trigrams)

66306

In [34]:
# choose a random sample from dictionary where values are the probabilities
def random_sample(d):
    r = random.random()
    cumulative = 0
    for w, p in iteritems(d):
        cumulative += p
        if r < cumulative:
            return w

### Test Spinner

Randomly take a review. Print original and spun review.

In [71]:
def test_spinner():
    review = random.choice(positive_reviews)
    s = review.text.lower()
    print"Original:\n", review
    tokens = nltk.tokenize.word_tokenize(s)
    for i in range(len(tokens) - 2):
        if random.random() < 0.3: # 30% chance of replacement
            k = (tokens[i], tokens[i+2])
            if k in trigram_probabilities:
                w = random_sample(trigram_probabilities[k])
                tokens[i+1] = w
    print("\nSpun:\n")
    print(" ".join(tokens).replace(" .", ".").replace(" '", "'").replace(" ,", ",").replace("$ ", "$").replace(" !", "!"))

In [72]:
if __name__ == '__main__':
    test_spinner()
    print("=-=-"*31)
    print('\n')
    test_spinner()

Original:
<review_text>
Kingston's 2GB Data Traveler USB drive is perfect to storage files. I have looked for a 2 GB USB and found this one to be the cheapest based on its characteristices. I think it is a bargain price
</review_text>

Spun:

kingston's 2gb data traveler usb drive is perfect to storage files. i first looked for a 2 gb usb and chose this one needs be the cheapest based on its job. i think it with a bargain price
=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-


Original:
<review_text>
My wife and I used the FR300 during the week-long Queens blackout this past summer and this little radio delivered. One 2 minute winding lasted for an hour on medium volume. The cell phone charger is cool, but it takes a lot of winding. The light is good in a pinch, but not particularly useful
</review_text>

Spun:

my wife and i recognize the fr300 during the week-long queens blackout this past, and my little rad